In [78]:
from proj1_helpers import load_csv_data, predict_labels, create_csv_submission
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import pandas as pd
import implementation as imp
import numpy as np
from created_helpers import batch_iter, least_squares_cost, sigmoid_fn, \
                            logistic_regression_cost, \
                            reg_logistic_regression_cost

In [79]:
data_path = './project_data/'

In [80]:
train_data_path = os.path.join(data_path, "train.csv")
test_data_path = os.path.join(data_path,"test.csv")

y_train, x_train, ids_train = load_csv_data(train_data_path)
y_test, x_test, ids_test = load_csv_data(test_data_path)

In [81]:
x_train

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

In [82]:
data_train = pd.read_csv(train_data_path)
data_test = pd.read_csv(test_data_path)
data_train.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


In [83]:
#TODO: explore dataset

In [84]:
#least_squares_GD 

random.seed(42)
initial_gamma = random.uniform(0.5, 1.5)
m = x_train.shape[1]
initial_w = np.array([0] * m)
max_iters = 22

#bug with range(max_iters) if import from implementation.py 
#TODO: debug 
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    '''
    :param y: labels -> m x 1 np.array
    :param tx: X -> m x n-1 np.array
    :param initial_w: weights -> n x 1 np.array
    :param max_iters: int
    :param gamma: step size -> float
    :return: (w, loss) -> (np.array, int)
    '''

    m = y.shape[0]
    for i in range(max_iters):
        error = np.dot(tx, initial_w) - y  # (m,1)
        gradient = np.dot(tx.T, error)  # (n,m)x(m,1) -> (n,1)
        initial_w = initial_w - (gamma / m) * gradient

    cost = least_squares_cost(y, tx, initial_w)
    return initial_w, cost

optimal_weight, loss = least_squares_GD(y_train, x_train, initial_w, max_iters, initial_gamma)

y_pred = predict_labels(optimal_weight, x_test)
y_pred

create_csv_submission(ids_test, y_pred,'least_square_gd.csv')

In [86]:
#least_squares_SGD 

random.seed(42)
initial_gamma = random.uniform(0.5, 1.5)
m = x_train.shape[1]
initial_w = np.array([0] * m)
max_iters = 22

import numpy as np
from created_helpers import batch_iter, least_squares_cost, sigmoid_fn, \
                            logistic_regression_cost, \
                            reg_logistic_regression_cost

def least_squares_SGD(y, tx, initial_w, max_iters, gamma):

    m = y.shape[0]
    for i in range(max_iters):
        if (i > 0):
            gamma = 1 / i
        for minibatch_y, minibatch_tx in batch_iter(y, tx, 1): # O(1) runtime
            error = np.dot(minibatch_tx, initial_w) - minibatch_y  # (1,1)
            gradient = np.dot(minibatch_tx.T, error)  # (n,1)x(1,1) -> (n,1)
            initial_w = initial_w - (gamma / m) * gradient

    cost = least_squares_cost(y, tx, initial_w)

    return initial_w, cost

optimal_weight, loss = least_squares_SGD(y_train, x_train, initial_w, max_iters, initial_gamma)

y_pred = predict_labels(optimal_weight, x_test)
y_pred

create_csv_submission(ids_test, y_pred,'least_square_sgd.csv')